In [10]:
!nvidia-smi

Wed Jul 19 19:55:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   4440MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [1]:
!pip install torch
!pip install bitsandbytes
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-_bhmpk8q/accelerate_293189525bc24852842c63781269dd34
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-_bhmpk8q/accelerate_293189525bc24852842c63781269dd34
  Resolved https://github.com/huggingface/accelerate.git to commit 505b5be044dbb8b348053758d6cddb5a2f882187
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-gpm8q5xe/transformers_2c5c76d89a474413b732ba03a84a8fa5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-gpm8q5xe/transformers_2c5c76d89a474413b732ba03a84a8fa5
  Resolved https://github.com/huggingface/transformers.git to commit 2bd7a27a671fd1d98059124024f580f8f5c0f3b5
  Installing build dependencies ... don

In [12]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [3]:
base_model_name_or_path = "VMware/open-llama-13b-open-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    low_cpu_mem_usage=True,
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Load the LoRA adapter

#lora_model_name_or_path = "Gustrd/open-llama-13b-lora-cabra-adapter"
#model = PeftModel.from_pretrained(model, lora_model_name_or_path)

#model.eval()

#if torch.__version__ >= "2":
#    model = torch.compile(model)

In [4]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

In [5]:
def instruct(
    instruction ,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=200,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    do_sample = temperature > 0.0
    
    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    splitted_generated_text = generated_text.split('\n')
    first_generated_text = splitted_generated_text[0]
    
    return first_generated_text

In [6]:
traducao = instruct("Translate the following text from english to portuguese.",input="He's never done anything.")

print(traducao)

/usr/local/lib/python3.9/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Ele nunca teve que fazer nada.


In [10]:
import pandas as pd

model_code = 'ol13oi'

#First time
opus_100_pt_validation = pd.read_parquet('https://huggingface.co/api/datasets/opus100/parquet/en-pt/validation/0.parquet')
#Continuing
#opus_100_pt_validation = pd.read_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')

In [8]:
max_string = ""

for json in opus_100_pt_validation.translation:
    
    string = json['en'] 
    if len(string) > len(max_string):
        max_string = string
        
    string = json['pt'] 
    if len(string) > len(max_string):
        max_string = string
        
print("The biggest string is: " + max_string)

tokens = tokenizer.tokenize(max_string)
num_tokens = len(tokens)

print("The maximum tokens number is: " + str(num_tokens))

The biggest string is: having regard to the Annual Report of the Court of Auditors on the implementation of the budget concerning the financial year 2011, together with the institutions’ replies [3], and to the Court of Auditors’ special reports,having regard to the statement of assurance [4] as to the reliability of the accounts and the legality and regularity of the underlying transactions provided by the Court of Auditors for the financial year 2011 pursuant to Article 287 of the Treaty on the Functioning of the European Union,having regard to the Council’s recommendation of 12 February 2013 on discharge to be given to the Commission in respect of the implementation of the budget for the financial year 2011 (05752/2013 – C7-0038/2013),
The maximum tokens number is: 181


In [ ]:
dataset = opus_100_pt_validation.translation

row_number = len(dataset)
current_row = 0

for json in opus_100_pt_validation.translation:
    current_row=current_row+1
    
    if model_code in json and json[model_code]:
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução existente: " + json[model_code] )
    else:
        originalString = json['en'] 
        traducao = instruct("Translate the following text from english to portuguese.",input=originalString)
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução: " + traducao)
        json[model_code]=traducao
        opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
    

1/2000 - Tradução: Ele nunca fez nada.
2/2000 - Tradução: Tenho medo da pandemia, mas estou a aceitar o risco.
3/2000 - Tradução: Você vai ser bom?
4/2000 - Tradução: Sim, estava a meu lado, não estava ao teu lado.
5/2000 - Tradução: Então você vai lá!
6/2000 - Tradução: Porque você não o wash.
7/2000 - Tradução: Parker's voice sounded a bit different on the telephone.
8/2000 - Tradução: Detective Bird, from the Rio de Janeiro police force, thinks that the following happened: 
9/2000 - Tradução: Se tivesse amado, você teria bancado.
10/2000 - Tradução: - Temos dois suspeitos.
11/2000 - Tradução: - Abrimos a porta...
12/2000 - Tradução: As graças às finanças, o governo pode oferecer um subsídio de interesse em dinheiro prestado.
13/2000 - Tradução: Não há nada. Assim, Mikey Molloy fez uma brincadeira com mim, fazendo passar por alucinatória, a ideia de ter um dólar.
14/2000 - Tradução: Juro a todos que esta tarefa será superada
15/2000 - Tradução: Mike estava à procura de você.
16/2000 

In [ ]:
opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')

In [48]:

saved_parquet = pd.read_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
print(saved_parquet.translation[200])                        

{'cabra0': None, 'en': 'To foster ethics in business, Camargo Corrêa Group elaborated the Camargo Corrêa Business Code of Conduct, which gathers the principles guiding the relationship of companies, executives, shareholders, and employees.', 'pt': 'Por incentivar a ética na condução dos negócios, o Grupo Camargo Corrêa elaborou o Código de Conduta Empresarial Camargo Corrêa, que reúne os princípios que norteiam o relacionamento de empresas, executivos, acionistas e profissionais.'}
